In [2]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
#! git clone https://github.com/AlexeyAB/darknet.git "/content/gdrive/My Drive/darknet"

In [3]:
!pwd
%cd /content/gdrive

/content
/content/gdrive


In [7]:
!ls

'My Drive'


In [4]:
%cd My Drive/darknet

/content/gdrive/My Drive/darknet


In [5]:
%%writefile Makefile
GPU=1
CUDNN=1
CUDNN_HALF=0
OPENCV=1
AVX=0
OPENMP=0
LIBSO=0
ZED_CAMERA=0 # ZED SDK 3.0 and above
ZED_CAMERA_v2_8=0 # ZED SDK 2.X

# set GPU=1 and CUDNN=1 to speedup on GPU
# set CUDNN_HALF=1 to further speedup 3 x times (Mixed-precision on Tensor Cores) GPU: Volta, Xavier, Turing and higher
# set AVX=1 and OPENMP=1 to speedup on CPU (if error occurs then set AVX=0)

USE_CPP=0
DEBUG=0

ARCH= -gencode arch=compute_30,code=sm_30 \
      -gencode arch=compute_35,code=sm_35 \
      -gencode arch=compute_50,code=[sm_50,compute_50] \
      -gencode arch=compute_52,code=[sm_52,compute_52] \
	    -gencode arch=compute_61,code=[sm_61,compute_61]

OS := $(shell uname)

# Tesla V100
# ARCH= -gencode arch=compute_70,code=[sm_70,compute_70]

# GeForce RTX 2080 Ti, RTX 2080, RTX 2070, Quadro RTX 8000, Quadro RTX 6000, Quadro RTX 5000, Tesla T4, XNOR Tensor Cores
# ARCH= -gencode arch=compute_75,code=[sm_75,compute_75]

# Jetson XAVIER
# ARCH= -gencode arch=compute_72,code=[sm_72,compute_72]

# GTX 1080, GTX 1070, GTX 1060, GTX 1050, GTX 1030, Titan Xp, Tesla P40, Tesla P4
# ARCH= -gencode arch=compute_61,code=sm_61 -gencode arch=compute_61,code=compute_61

# GP100/Tesla P100 - DGX-1
# ARCH= -gencode arch=compute_60,code=sm_60

# For Jetson TX1, Tegra X1, DRIVE CX, DRIVE PX - uncomment:
# ARCH= -gencode arch=compute_53,code=[sm_53,compute_53]

# For Jetson Tx2 or Drive-PX2 uncomment:
# ARCH= -gencode arch=compute_62,code=[sm_62,compute_62]


VPATH=./src/
EXEC=darknet
OBJDIR=./obj/

ifeq ($(LIBSO), 1)
LIBNAMESO=libdarknet.so
APPNAMESO=uselib
endif

ifeq ($(USE_CPP), 1)
CC=g++
else
CC=gcc
endif

CPP=g++ -std=c++11
NVCC=nvcc
OPTS=-Ofast
LDFLAGS= -lm -pthread
COMMON= -Iinclude/ -I3rdparty/stb/include
CFLAGS=-Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC

ifeq ($(DEBUG), 1)
#OPTS= -O0 -g
#OPTS= -Og -g
COMMON+= -DDEBUG
CFLAGS+= -DDEBUG
else
ifeq ($(AVX), 1)
CFLAGS+= -ffp-contract=fast -mavx -mavx2 -msse3 -msse4.1 -msse4.2 -msse4a
endif
endif

CFLAGS+=$(OPTS)

ifneq (,$(findstring MSYS_NT,$(OS)))
LDFLAGS+=-lws2_32
endif

ifeq ($(OPENCV), 1)
COMMON+= -DOPENCV
CFLAGS+= -DOPENCV
LDFLAGS+= `pkg-config --libs opencv4 2> /dev/null || pkg-config --libs opencv`
COMMON+= `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv`
endif

ifeq ($(OPENMP), 1)
CFLAGS+= -fopenmp
LDFLAGS+= -lgomp
endif

ifeq ($(GPU), 1)
COMMON+= -DGPU -I/usr/local/cuda/include/
CFLAGS+= -DGPU
ifeq ($(OS),Darwin) #MAC
LDFLAGS+= -L/usr/local/cuda/lib -lcuda -lcudart -lcublas -lcurand
else
LDFLAGS+= -L/usr/local/cuda/lib64 -lcuda -lcudart -lcublas -lcurand
endif
endif

ifeq ($(CUDNN), 1)
COMMON+= -DCUDNN
ifeq ($(OS),Darwin) #MAC
CFLAGS+= -DCUDNN -I/usr/local/cuda/include
LDFLAGS+= -L/usr/local/cuda/lib -lcudnn
else
CFLAGS+= -DCUDNN -I/usr/local/cudnn/include
LDFLAGS+= -L/usr/local/cudnn/lib64 -lcudnn
endif
endif

ifeq ($(CUDNN_HALF), 1)
COMMON+= -DCUDNN_HALF
CFLAGS+= -DCUDNN_HALF
ARCH+= -gencode arch=compute_70,code=[sm_70,compute_70]
endif

ifeq ($(ZED_CAMERA), 1)
CFLAGS+= -DZED_STEREO -I/usr/local/zed/include
ifeq ($(ZED_CAMERA_v2_8), 1)
LDFLAGS+= -L/usr/local/zed/lib -lsl_core -lsl_input -lsl_zed
#-lstdc++ -D_GLIBCXX_USE_CXX11_ABI=0
else
LDFLAGS+= -L/usr/local/zed/lib -lsl_zed
#-lstdc++ -D_GLIBCXX_USE_CXX11_ABI=0
endif
endif

OBJ=image_opencv.o http_stream.o gemm.o utils.o dark_cuda.o convolutional_layer.o list.o image.o activations.o im2col.o col2im.o blas.o crop_layer.o dropout_layer.o maxpool_layer.o softmax_layer.o data.o matrix.o network.o connected_layer.o cost_layer.o parser.o option_list.o darknet.o detection_layer.o captcha.o route_layer.o writing.o box.o nightmare.o normalization_layer.o avgpool_layer.o coco.o dice.o yolo.o detector.o layer.o compare.o classifier.o local_layer.o swag.o shortcut_layer.o activation_layer.o rnn_layer.o gru_layer.o rnn.o rnn_vid.o crnn_layer.o demo.o tag.o cifar.o go.o batchnorm_layer.o art.o region_layer.o reorg_layer.o reorg_old_layer.o super.o voxel.o tree.o yolo_layer.o gaussian_yolo_layer.o upsample_layer.o lstm_layer.o conv_lstm_layer.o scale_channels_layer.o sam_layer.o
ifeq ($(GPU), 1)
LDFLAGS+= -lstdc++
OBJ+=convolutional_kernels.o activation_kernels.o im2col_kernels.o col2im_kernels.o blas_kernels.o crop_layer_kernels.o dropout_layer_kernels.o maxpool_layer_kernels.o network_kernels.o avgpool_layer_kernels.o
endif

OBJS = $(addprefix $(OBJDIR), $(OBJ))
DEPS = $(wildcard src/*.h) Makefile include/darknet.h

all: $(OBJDIR) backup results setchmod $(EXEC) $(LIBNAMESO) $(APPNAMESO)

ifeq ($(LIBSO), 1)
CFLAGS+= -fPIC

$(LIBNAMESO): $(OBJDIR) $(OBJS) include/yolo_v2_class.hpp src/yolo_v2_class.cpp
	$(CPP) -shared -std=c++11 -fvisibility=hidden -DLIB_EXPORTS $(COMMON) $(CFLAGS) $(OBJS) src/yolo_v2_class.cpp -o $@ $(LDFLAGS)

$(APPNAMESO): $(LIBNAMESO) include/yolo_v2_class.hpp src/yolo_console_dll.cpp
	$(CPP) -std=c++11 $(COMMON) $(CFLAGS) -o $@ src/yolo_console_dll.cpp $(LDFLAGS) -L ./ -l:$(LIBNAMESO)
endif

$(EXEC): $(OBJS)
	$(CPP) -std=c++11 $(COMMON) $(CFLAGS) $^ -o $@ $(LDFLAGS)

$(OBJDIR)%.o: %.c $(DEPS)
	$(CC) $(COMMON) $(CFLAGS) -c $< -o $@

$(OBJDIR)%.o: %.cpp $(DEPS)
	$(CPP) -std=c++11 $(COMMON) $(CFLAGS) -c $< -o $@

$(OBJDIR)%.o: %.cu $(DEPS)
	$(NVCC) $(ARCH) $(COMMON) --compiler-options "$(CFLAGS)" -c $< -o $@

$(OBJDIR):
	mkdir -p $(OBJDIR)
backup:
	mkdir -p backup
results:
	mkdir -p results
setchmod:
	chmod +x *.sh

.PHONY: clean

clean:
	rm -rf $(OBJS) $(EXEC) $(LIBNAMESO) $(APPNAMESO)

Overwriting Makefile


In [6]:
!ls
!make

3rdparty	    DarknetConfig.cmake.in  obj
backup		    darknet.py		    predictions.jpg
bad_label.list	    darknet_video.py	    README.md
bad.list	    data		    results
build		    dog.data		    scripts
build.ps1	    dog.name		    src
build.sh	    image_yolov2.sh	    video_v2.sh
cfg		    image_yolov3.sh	    video_yolov3.sh
chart.png	    include		    yolov3-voc.cfg
chart_yolo-dog.png  json_mjpeg_streams.sh   yolov3.weights
cmake		    LICENSE		    yolov3.weights.1
CMakeLists.txt	    Makefile		    yolov3.weights.2
darknet		    net_cam_v3.sh	    yolov3.weights.3
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**

In [7]:
!pwd
!./darknet

/content/gdrive/My Drive/darknet
usage: ./darknet <function>


In [0]:
|! wget https://pjreddie.com/media/files/yolov3.weights

--2020-06-08 07:03:39--  https://pjreddie.com/media/files/yolov3.weights
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248007048 (237M) [application/octet-stream]
Saving to: ‘yolov3.weights.3’

yolov3.weights.3      0%[                    ] 167.70K   129KB/s               ^C


In [0]:
! ./darknet detect cfg/yolov3.cfg yolov3.weights cfg/task/datasets/dog.4.jpg

 CUDA-version: 10010 (10010), cuDNN: 7.6.5, GPU count: 1  
 OpenCV version: 3.2.0


KeyboardInterrupt: ignored

In [0]:
!sudo apt install libopencv-dev

In [0]:
#%cd ..
#%cd Darknet3


In [0]:
!pwd

/content/gdrive/My Drive/darknet/cfg/task


In [0]:
%pycat cfg/yolo-dog.cfg

In [0]:
%%writefile cfg/yolo-dog.cfg
[net]
# Testing
#batch=1
#subdivisions=1
# Training
batch=64
subdivisions=16
width=416
height=416
channels=3
momentum=0.949
decay=0.0005
angle=0
saturation = 1.5
exposure = 1.5
hue=.1

learning_rate=0.001
burn_in=1000
max_batches = 6000
policy=steps
steps=4800,5400 
scales=.1,.1

#cutmix=1
mosaic=1

#:104x104 54:52x52 85:26x26 104:13x13 for 416

[convolutional]
batch_normalize=1
filters=32
size=3
stride=1
pad=1
activation=mish

# Downsample

[convolutional]
batch_normalize=1
filters=64
size=3
stride=2
pad=1
activation=mish

[convolutional]
batch_normalize=1
filters=64
size=1
stride=1
pad=1
activation=mish

[route]
layers = -2

[convolutional]
batch_normalize=1
filters=64
size=1
stride=1
pad=1
activation=mish

[convolutional]
batch_normalize=1
filters=32
size=1
stride=1
pad=1
activation=mish

[convolutional]
batch_normalize=1
filters=64
size=3
stride=1
pad=1
activation=mish

[shortcut]
from=-3
activation=linear

[convolutional]
batch_normalize=1
filters=64
size=1
stride=1
pad=1
activation=mish

[route]
layers = -1,-7

[convolutional]
batch_normalize=1
filters=64
size=1
stride=1
pad=1
activation=mish

# Downsample

[convolutional]
batch_normalize=1
filters=128
size=3
stride=2
pad=1
activation=mish

[convolutional]
batch_normalize=1
filters=64
size=1
stride=1
pad=1
activation=mish

[route]
layers = -2

[convolutional]
batch_normalize=1
filters=64
size=1
stride=1
pad=1
activation=mish

[convolutional]
batch_normalize=1
filters=64
size=1
stride=1
pad=1
activation=mish

[convolutional]
batch_normalize=1
filters=64
size=3
stride=1
pad=1
activation=mish

[shortcut]
from=-3
activation=linear

[convolutional]
batch_normalize=1
filters=64
size=1
stride=1
pad=1
activation=mish

[convolutional]
batch_normalize=1
filters=64
size=3
stride=1
pad=1
activation=mish

[shortcut]
from=-3
activation=linear

[convolutional]
batch_normalize=1
filters=64
size=1
stride=1
pad=1
activation=mish

[route]
layers = -1,-10

[convolutional]
batch_normalize=1
filters=128
size=1
stride=1
pad=1
activation=mish

# Downsample

[convolutional]
batch_normalize=1
filters=256
size=3
stride=2
pad=1
activation=mish

[convolutional]
batch_normalize=1
filters=128
size=1
stride=1
pad=1
activation=mish

[route]
layers = -2

[convolutional]
batch_normalize=1
filters=128
size=1
stride=1
pad=1
activation=mish

[convolutional]
batch_normalize=1
filters=128
size=1
stride=1
pad=1
activation=mish

[convolutional]
batch_normalize=1
filters=128
size=3
stride=1
pad=1
activation=mish

[shortcut]
from=-3
activation=linear

[convolutional]
batch_normalize=1
filters=128
size=1
stride=1
pad=1
activation=mish

[convolutional]
batch_normalize=1
filters=128
size=3
stride=1
pad=1
activation=mish

[shortcut]
from=-3
activation=linear

[convolutional]
batch_normalize=1
filters=128
size=1
stride=1
pad=1
activation=mish

[convolutional]
batch_normalize=1
filters=128
size=3
stride=1
pad=1
activation=mish

[shortcut]
from=-3
activation=linear

[convolutional]
batch_normalize=1
filters=128
size=1
stride=1
pad=1
activation=mish

[convolutional]
batch_normalize=1
filters=128
size=3
stride=1
pad=1
activation=mish

[shortcut]
from=-3
activation=linear


[convolutional]
batch_normalize=1
filters=128
size=1
stride=1
pad=1
activation=mish

[convolutional]
batch_normalize=1
filters=128
size=3
stride=1
pad=1
activation=mish

[shortcut]
from=-3
activation=linear

[convolutional]
batch_normalize=1
filters=128
size=1
stride=1
pad=1
activation=mish

[convolutional]
batch_normalize=1
filters=128
size=3
stride=1
pad=1
activation=mish

[shortcut]
from=-3
activation=linear

[convolutional]
batch_normalize=1
filters=128
size=1
stride=1
pad=1
activation=mish

[convolutional]
batch_normalize=1
filters=128
size=3
stride=1
pad=1
activation=mish

[shortcut]
from=-3
activation=linear

[convolutional]
batch_normalize=1
filters=128
size=1
stride=1
pad=1
activation=mish

[convolutional]
batch_normalize=1
filters=128
size=3
stride=1
pad=1
activation=mish

[shortcut]
from=-3
activation=linear

[convolutional]
batch_normalize=1
filters=128
size=1
stride=1
pad=1
activation=mish

[route]
layers = -1,-28

[convolutional]
batch_normalize=1
filters=256
size=1
stride=1
pad=1
activation=mish

# Downsample

[convolutional]
batch_normalize=1
filters=512
size=3
stride=2
pad=1
activation=mish

[convolutional]
batch_normalize=1
filters=256
size=1
stride=1
pad=1
activation=mish

[route]
layers = -2

[convolutional]
batch_normalize=1
filters=256
size=1
stride=1
pad=1
activation=mish

[convolutional]
batch_normalize=1
filters=256
size=1
stride=1
pad=1
activation=mish

[convolutional]
batch_normalize=1
filters=256
size=3
stride=1
pad=1
activation=mish

[shortcut]
from=-3
activation=linear


[convolutional]
batch_normalize=1
filters=256
size=1
stride=1
pad=1
activation=mish

[convolutional]
batch_normalize=1
filters=256
size=3
stride=1
pad=1
activation=mish

[shortcut]
from=-3
activation=linear


[convolutional]
batch_normalize=1
filters=256
size=1
stride=1
pad=1
activation=mish

[convolutional]
batch_normalize=1
filters=256
size=3
stride=1
pad=1
activation=mish

[shortcut]
from=-3
activation=linear


[convolutional]
batch_normalize=1
filters=256
size=1
stride=1
pad=1
activation=mish

[convolutional]
batch_normalize=1
filters=256
size=3
stride=1
pad=1
activation=mish

[shortcut]
from=-3
activation=linear


[convolutional]
batch_normalize=1
filters=256
size=1
stride=1
pad=1
activation=mish

[convolutional]
batch_normalize=1
filters=256
size=3
stride=1
pad=1
activation=mish

[shortcut]
from=-3
activation=linear


[convolutional]
batch_normalize=1
filters=256
size=1
stride=1
pad=1
activation=mish

[convolutional]
batch_normalize=1
filters=256
size=3
stride=1
pad=1
activation=mish

[shortcut]
from=-3
activation=linear


[convolutional]
batch_normalize=1
filters=256
size=1
stride=1
pad=1
activation=mish

[convolutional]
batch_normalize=1
filters=256
size=3
stride=1
pad=1
activation=mish

[shortcut]
from=-3
activation=linear

[convolutional]
batch_normalize=1
filters=256
size=1
stride=1
pad=1
activation=mish

[convolutional]
batch_normalize=1
filters=256
size=3
stride=1
pad=1
activation=mish

[shortcut]
from=-3
activation=linear

[convolutional]
batch_normalize=1
filters=256
size=1
stride=1
pad=1
activation=mish

[route]
layers = -1,-28

[convolutional]
batch_normalize=1
filters=512
size=1
stride=1
pad=1
activation=mish

# Downsample

[convolutional]
batch_normalize=1
filters=1024
size=3
stride=2
pad=1
activation=mish

[convolutional]
batch_normalize=1
filters=512
size=1
stride=1
pad=1
activation=mish

[route]
layers = -2

[convolutional]
batch_normalize=1
filters=512
size=1
stride=1
pad=1
activation=mish

[convolutional]
batch_normalize=1
filters=512
size=1
stride=1
pad=1
activation=mish

[convolutional]
batch_normalize=1
filters=512
size=3
stride=1
pad=1
activation=mish

[shortcut]
from=-3
activation=linear

[convolutional]
batch_normalize=1
filters=512
size=1
stride=1
pad=1
activation=mish

[convolutional]
batch_normalize=1
filters=512
size=3
stride=1
pad=1
activation=mish

[shortcut]
from=-3
activation=linear

[convolutional]
batch_normalize=1
filters=512
size=1
stride=1
pad=1
activation=mish

[convolutional]
batch_normalize=1
filters=512
size=3
stride=1
pad=1
activation=mish

[shortcut]
from=-3
activation=linear

[convolutional]
batch_normalize=1
filters=512
size=1
stride=1
pad=1
activation=mish

[convolutional]
batch_normalize=1
filters=512
size=3
stride=1
pad=1
activation=mish

[shortcut]
from=-3
activation=linear

[convolutional]
batch_normalize=1
filters=512
size=1
stride=1
pad=1
activation=mish

[route]
layers = -1,-16

[convolutional]
batch_normalize=1
filters=1024
size=1
stride=1
pad=1
activation=mish
stopbackward=800

##########################

[convolutional]
batch_normalize=1
filters=512
size=1
stride=1
pad=1
activation=leaky

[convolutional]
batch_normalize=1
size=3
stride=1
pad=1
filters=1024
activation=leaky

[convolutional]
batch_normalize=1
filters=512
size=1
stride=1
pad=1
activation=leaky

### SPP ###
[maxpool]
stride=1
size=5

[route]
layers=-2

[maxpool]
stride=1
size=9

[route]
layers=-4

[maxpool]
stride=1
size=13

[route]
layers=-1,-3,-5,-6
### End SPP ###

[convolutional]
batch_normalize=1
filters=512
size=1
stride=1
pad=1
activation=leaky

[convolutional]
batch_normalize=1
size=3
stride=1
pad=1
filters=1024
activation=leaky

[convolutional]
batch_normalize=1
filters=512
size=1
stride=1
pad=1
activation=leaky

[convolutional]
batch_normalize=1
filters=256
size=1
stride=1
pad=1
activation=leaky

[upsample]
stride=2

[route]
layers = 85

[convolutional]
batch_normalize=1
filters=256
size=1
stride=1
pad=1
activation=leaky

[route]
layers = -1, -3

[convolutional]
batch_normalize=1
filters=256
size=1
stride=1
pad=1
activation=leaky

[convolutional]
batch_normalize=1
size=3
stride=1
pad=1
filters=512
activation=leaky

[convolutional]
batch_normalize=1
filters=256
size=1
stride=1
pad=1
activation=leaky

[convolutional]
batch_normalize=1
size=3
stride=1
pad=1
filters=512
activation=leaky

[convolutional]
batch_normalize=1
filters=256
size=1
stride=1
pad=1
activation=leaky

[convolutional]
batch_normalize=1
filters=128
size=1
stride=1
pad=1
activation=leaky

[upsample]
stride=2

[route]
layers = 54

[convolutional]
batch_normalize=1
filters=128
size=1
stride=1
pad=1
activation=leaky

[route]
layers = -1, -3

[convolutional]
batch_normalize=1
filters=128
size=1
stride=1
pad=1
activation=leaky

[convolutional]
batch_normalize=1
size=3
stride=1
pad=1
filters=256
activation=leaky

[convolutional]
batch_normalize=1
filters=128
size=1
stride=1
pad=1
activation=leaky

[convolutional]
batch_normalize=1
size=3
stride=1
pad=1
filters=256
activation=leaky

[convolutional]
batch_normalize=1
filters=128
size=1
stride=1
pad=1
activation=leaky

##########################

[convolutional]
batch_normalize=1
size=3
stride=1
pad=1
filters=256
activation=leaky

[convolutional]
size=1
stride=1
pad=1
filters=24
activation=linear


[yolo]
mask = 0,1,2
anchors = 12, 16, 19, 36, 40, 28, 36, 75, 76, 55, 72, 146, 142, 110, 192, 243, 459, 401
classes=3
num=9
jitter=.3
ignore_thresh = .7
truth_thresh = 1
scale_x_y = 1.2
iou_thresh=0.213
cls_normalizer=1.0
iou_normalizer=0.07
iou_loss=ciou
nms_kind=greedynms
beta_nms=0.6
max_delta=5


[route]
layers = -4

[convolutional]
batch_normalize=1
size=3
stride=2
pad=1
filters=256
activation=leaky

[route]
layers = -1, -16

[convolutional]
batch_normalize=1
filters=256
size=1
stride=1
pad=1
activation=leaky

[convolutional]
batch_normalize=1
size=3
stride=1
pad=1
filters=512
activation=leaky

[convolutional]
batch_normalize=1
filters=256
size=1
stride=1
pad=1
activation=leaky

[convolutional]
batch_normalize=1
size=3
stride=1
pad=1
filters=512
activation=leaky

[convolutional]
batch_normalize=1
filters=256
size=1
stride=1
pad=1
activation=leaky

[convolutional]
batch_normalize=1
size=3
stride=1
pad=1
filters=512
activation=leaky

[convolutional]
size=1
stride=1
pad=1
filters=24
activation=linear


[yolo]
mask = 3,4,5
anchors = 12, 16, 19, 36, 40, 28, 36, 75, 76, 55, 72, 146, 142, 110, 192, 243, 459, 401
classes=3
num=9
jitter=.3
ignore_thresh = .7
truth_thresh = 1
scale_x_y = 1.1
iou_thresh=0.213
cls_normalizer=1.0
iou_normalizer=0.07
iou_loss=ciou
nms_kind=greedynms
beta_nms=0.6
max_delta=5


[route]
layers = -4

[convolutional]
batch_normalize=1
size=3
stride=2
pad=1
filters=512
activation=leaky

[route]
layers = -1, -37

[convolutional]
batch_normalize=1
filters=512
size=1
stride=1
pad=1
activation=leaky

[convolutional]
batch_normalize=1
size=3
stride=1
pad=1
filters=1024
activation=leaky

[convolutional]
batch_normalize=1
filters=512
size=1
stride=1
pad=1
activation=leaky

[convolutional]
batch_normalize=1
size=3
stride=1
pad=1
filters=1024
activation=leaky

[convolutional]
batch_normalize=1
filters=512
size=1
stride=1
pad=1
activation=leaky

[convolutional]
batch_normalize=1
size=3
stride=1
pad=1
filters=1024
activation=leaky

[convolutional]
size=1
stride=1
pad=1
filters=24
activation=linear


[yolo]
mask = 6,7,8
anchors = 12, 16, 19, 36, 40, 28, 36, 75, 76, 55, 72, 146, 142, 110, 192, 243, 459, 401
classes=3
num=9
jitter=.3
ignore_thresh = .7
truth_thresh = 1
random=1
scale_x_y = 1.05
iou_thresh=0.213
cls_normalizer=1.0
iou_normalizer=0.07
iou_loss=ciou
nms_kind=greedynms
beta_nms=0.6
max_delta=5

Overwriting cfg/yolo-dog.cfg


In [0]:
!pwd

/content/gdrive/My Drive/darknet


In [0]:
%pycat build/darknet/x64/train.txt

In [21]:
%%writefile build/darknet/x64/data/train.txt

/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.141.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.142.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.140.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.139.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.144.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.136.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.137.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.138.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.135.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.143.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.134.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.133.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.130.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.131.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.132.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.124.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.125.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.126.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.127.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.128.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.129.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.122.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.123.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.118.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.119.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.117.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.121.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.112.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.114.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.115.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.116.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.113.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.120.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.111.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.110.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.105.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.104.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.107.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.106.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.103.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.100.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.102.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.109.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.101.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.108.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.177.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.184.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.182.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.183.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.185.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.181.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.18.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.180.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.179.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.178.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.172.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.173.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.175.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.174.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.17.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.170.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.171.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.169.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.176.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.168.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.167.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.164.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.162.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.165.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.163.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.166.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.160.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.16.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.159.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.161.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.158.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.157.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.152.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.154.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.153.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.156.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.147.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.148.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.150.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.151.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.149.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.155.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.146.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.145.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.261.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.258.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.259.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.260.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.253.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.254.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.256.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.255.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.257.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.252.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.26.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.251.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.250.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.242.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.245.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.247.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.248.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.246.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.241.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.243.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.244.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.25.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.240.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.249.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.24.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.231.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.232.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.236.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.238.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.234.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.235.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.233.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.237.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.239.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.230.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.23.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.226.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.224.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.227.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.225.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.229.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.223.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.228.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.221.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.222.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.220.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.22.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.219.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.211.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.215.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.216.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.214.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.213.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.218.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.21.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.210.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.212.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.217.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.209.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.208.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.201.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.200.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.203.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.204.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.205.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.202.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.207.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.199.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.20.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.206.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.198.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.197.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.192.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.193.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.196.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.195.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.188.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.187.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.189.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.19.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.191.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.190.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.194.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.186.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.42.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.419.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.418.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.411.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.413.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.414.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.416.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.415.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.417.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.412.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.410.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.41.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.407.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.409.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.402.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.401.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.403.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.404.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.405.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.406.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.400.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.408.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.40.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.399.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.395.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.396.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.398.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.39.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.390.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.391.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.392.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.393.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.394.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.397.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.389.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.388.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.38.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.37.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.36.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.387.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.32.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.33.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.34.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.35.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.31.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.386.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.30.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.294.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.29.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.291.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.289.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.290.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.293.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.285.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.287.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.288.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.286.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.292.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.284.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.283.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.280.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.28.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.279.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.282.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.275.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.276.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.278.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.277.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.274.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.281.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.273.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.272.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.27.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.269.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.268.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.267.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.271.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.264.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.265.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.266.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.263.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.270.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.262.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.445.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.444.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.443.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.43.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.442.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.441.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.440.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.44.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.439.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.438.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.429.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.424.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.426.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.427.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.425.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.428.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.421.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.420.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.423.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.422.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.84.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.76.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.75.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.70.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.69.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.74.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.65.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.67.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.68.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.72.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.71.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.66.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.73.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.64.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.63.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.61.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.59.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.58.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.60.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.55.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.54.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.57.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.62.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.56.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.53.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.52.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.496.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.497.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.495.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.51.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.490.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.492.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.491.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.493.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.494.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.50.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.49.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.489.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.484.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.486.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.485.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.488.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.48.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.480.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.482.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.483.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.481.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.487.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.479.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.478.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.472.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.475.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.473.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.474.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.477.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.469.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.47.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.470.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.471.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.476.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.468.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.467.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.462.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.461.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.465.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.464.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.463.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.459.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.46.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.460.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.466.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.458.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.457.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.453.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.454.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.452.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.451.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.456.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.45.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.448.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.449.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.450.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.455.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.447.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.446.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.430.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.431.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.433.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.434.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.435.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.436.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.437.jpg


Overwriting build/darknet/x64/data/train.txt


In [0]:
%%writefile build/darknet/x64/data/test.txt
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.432.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.99.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.98.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.97.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.93.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.94.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.96.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.89.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.90(1).jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.88.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.91.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.92.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.90.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.95.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.87.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.86.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.84(1).jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.82.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.83.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.85.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.77.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.79.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.80.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.81.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.78.jpg

Overwriting build/darknet/x64/data/test.txt


Overwriting dog.name


In [0]:
!ls

3rdparty	darknet			include		       scripts
backup		DarknetConfig.cmake.in	json_mjpeg_streams.sh  src
bad.list	darknet.py		LICENSE		       video_v2.sh
build		darknet_video.py	Makefile	       video_yolov3.sh
build.ps1	data			net_cam_v3.sh	       yolov3-voc.cfg
build.sh	dog.data		obj		       yolov3.weights
cfg		dog.name		predictions.jpg        yolov3.weights.1
cmake		image_yolov2.sh		README.md	       yolov3.weights.2
CMakeLists.txt	image_yolov3.sh		results


In [0]:
 !pwd

/content/gdrive/My Drive/darknet


In [0]:
#!cp cfg/task/classes.txt /data/names.list

cp: cannot create regular file '/data/names.list': No such file or directory


In [0]:
!pwd
%cd ..
%cd ..

/content/gdrive/My Drive/darknet/cfg/task


In [0]:
%%writefile cfg/task/datasets.data

classes= 3
train=/content/gdrive/My Drive/darknet/cfg/task/train.txt
valid=/content/gdrive/My Drive/darknet/cfg/task/test.txt
backup=/content/gdrive/My Drive/darknet/cfg/task/backup/ 

Overwriting cfg/task/datasets.data


In [0]:
!pwd
!ls cfg/task/datasets

In [17]:
!pwd
#%cd build/darknet/x64

/content/gdrive/My Drive/darknet


In [0]:
%pycat build/darknet/x64/datadivide/test.txt

In [13]:
%%writefile build/darknet/x64/datadivide/train.txt
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.29.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.30.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.28.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.27.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.26.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.31.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.24.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.25.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.23.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.22.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.21.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.13.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.14.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.15.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.16.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.17.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.18.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.19.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.12.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.20.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.11.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.9.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.10.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.8.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.4.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.7.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.5.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.6.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.3.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.2.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.73.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.66.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.68.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.70.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.71.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.72.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.69.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.67.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.74.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.65.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.64.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.62.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.60.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.59.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.61.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.55.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.56.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.57.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.58.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.63.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.54.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.53.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.46.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.47.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.48.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.49.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.50.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.52.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.51.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.45.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.44.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.43.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.40.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.39.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.38.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.42.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.33.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.35.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.36.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.37.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.34.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.32.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.41.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.97.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.96.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.95.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.92.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.91.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.93.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.94.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.90(1).jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.87.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.90.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.89.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.88.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.86.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.85.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.77.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.78.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.79.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.80.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.81.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.82.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.84.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.83.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.84(1).jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.164.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.158.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.159.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.165.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.149.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.161.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.160.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.148.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.162.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.163.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.104.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.110.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.138.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.139.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.105.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.111.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.113.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.107.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.106.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.112.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.116.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.102.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.103.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.117.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.129.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.101.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.115.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.114.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.100.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.128.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.125.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.131.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.119.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.118.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.75.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.130.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.124.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.132.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.126.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.76.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.127.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.133.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.137.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.123.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.99.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.98.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.122.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.136.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.108.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.120.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.134.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.135.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.109.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.121.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.146.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.152.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.153.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.147.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.145.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.151.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.144.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.150.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.154.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.140.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.141.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.155.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.143.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.157.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.156.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.142.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.175.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.166.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.243.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.245.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.246.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.244.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.248.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.239.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.240.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.242.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.241.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.247.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.238.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.237.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.227.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.228.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.229.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.230.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.234.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.232.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.233.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.231.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.236.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.226.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.235.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.225.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.221.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.222.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.220.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.224.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.215.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.217.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.218.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.219.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.216.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.223.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.214.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.213.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.210.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.206.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.205.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.208.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.207.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.209.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.212.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.204.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.203.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.211.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.202.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.201.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.197.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.198.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.196.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.200.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.191.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.192.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.193.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.194.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.195.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.199.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.190.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.189.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.184.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.183.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.185.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.186.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.188.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.179.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.180.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.181.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.182.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.187.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.178.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.177.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.168.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.167.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.169.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.170.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.171.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.173.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.172.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.174.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.176.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.256.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.257.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.258.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.255.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.253.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.252.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.254.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.251.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.250.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.249.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.285.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.279.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.284.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.283.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.274.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.276.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.277.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.278.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.275.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.273.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.272.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.269.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.270.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.271.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.267.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.268.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.263.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.264.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.265.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.266.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.262.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.261.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.259.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.260.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.316.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.317.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.318.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.315.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.314.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.309.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.311.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.312.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.310.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.313.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.307.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.308.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.304.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.305.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.306.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.303.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.302.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.301.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.299.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.300.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.294.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.295.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.296.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.297.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.290.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.292.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.293.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.291.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.298.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.289.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.288.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.287.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.286.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.280.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.281.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.282.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.338.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.330.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.331.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.332.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.336.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.334.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.333.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.335.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.329.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.337.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.327.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.328.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.326.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.325.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.319.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.320.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.321.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.322.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.324.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.323.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.359.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.360.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.356.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.355.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.350.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.352.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.353.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.354.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.357.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.351.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.358.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.349.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.348.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.347.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.346.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.341.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.343.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.340.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.342.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.345.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.339.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.344.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.377.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.378.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.376.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.375.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.374.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.370.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.372.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.371.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.369.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.373.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.368.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.365.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.366.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.364.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.367.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.362.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.363.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.361.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.428.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.430.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.429.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.427.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.435.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.426.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.425.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.421.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.422.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.419.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.424.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.416.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.415.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.418.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.420.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.417.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.423.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.414.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.413.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.409.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.410.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.412.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.411.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.407.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.408.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.401.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.404.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.402.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.403.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.405.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.406.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.400.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.399.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.398.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.397.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.390.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.394.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.391.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.393.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.392.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.389.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.396.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.395.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.385.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.387.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.388.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.386.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.384.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.383.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.380.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.382.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.381.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.379.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.473.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.472.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.464.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.463.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.465.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.470.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.469.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.467.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.466.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.468.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.471.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.462.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.461.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.456.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.457.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.454.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.455.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.460.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.452.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.453.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.458.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.451.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.459.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.450.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.449.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.440.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.441.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.442.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.443.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.446.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.444.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.445.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.439.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.448.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.447.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.438.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.437.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.432.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.431.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.433.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.434.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.436.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.555.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.554.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.550.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.551.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.549.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.553.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.545.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.544.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.547.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.548.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.546.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.552.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.543.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.542.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.537.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.539.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.540.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.538.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.534.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.535.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.536.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.533.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.541.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.532.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.531.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.524.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.525.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.526.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.527.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.528.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.530.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.521.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.522.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.523.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.529.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.520.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.519.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.515.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.517.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.516.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.510.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.511.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.512.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.513.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.514.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.518.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.509.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.508.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.504.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.506.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.507.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.505.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.503.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.502.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.499.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.501.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.500.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.497.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.494.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.496.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.495.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.490.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.491.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.493.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.492.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.498.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.489.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.488.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.487.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.486.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.483.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.485.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.482.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.480.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.479.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.481.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.484.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.474.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.475.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.476.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.477.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.478.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.616.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.617.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.618.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.619.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.615.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.623.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.614.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.613.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.609.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.607.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.610.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.608.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.612.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.603.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.605.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.606.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.604.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.611.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.602.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.601.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.595.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.598.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.597.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.596.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.600.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.592.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.593.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.594.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.591.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.599.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.590.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.589.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.583.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.584.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.586.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.585.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.588.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.580.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.581.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.582.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.579.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.587.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.578.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.577.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.573.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.574.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.575.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.572.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.569.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.568.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.571.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.570.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.576.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.567.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.566.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.560.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.561.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.562.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.563.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.565.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.564.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.556.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.558.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.559.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.557.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.707.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.702.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.703.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.704.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.701.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.700.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.706.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.697.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.699.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.698.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.705.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.696.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.695.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.686.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.687.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.688.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.689.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.690.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.692.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.691.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.685.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.694.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.684.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.693.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.683.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.681.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.678.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.680.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.675.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.674.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.676.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.677.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.679.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.682.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.673.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.672.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.668.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.669.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.667.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.671.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.662.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.664.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.664(1).jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.666.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.994.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.995.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.992.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.999.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.990.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.989.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.986.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.985.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.988.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.980.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.982.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.983.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.984.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.981.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.979.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.987.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.978.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.977.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.973.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.974.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.972.jpg

Overwriting build/darknet/x64/datadivide/train.txt


In [12]:
%%writefile build/darknet/x64/datadivide/train.txt
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.29.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.30.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.28.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.27.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.26.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.31.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.24.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.25.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.23.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.22.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.21.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.13.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.14.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.15.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.16.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.17.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.18.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.19.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.12.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.20.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.11.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.9.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.10.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.8.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.4.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.7.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.5.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.6.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.3.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.2.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.73.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.66.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.68.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.70.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.71.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.72.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.69.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.67.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.74.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.65.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.64.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.62.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.60.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.59.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.61.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.55.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.56.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.57.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.58.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.63.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.54.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.53.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.46.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.47.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.48.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.49.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.50.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.52.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.51.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.45.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.44.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.43.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.40.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.39.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.38.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.42.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.33.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.35.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.36.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.37.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.34.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.32.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.41.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.97.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.96.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.95.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.92.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.91.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.93.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.94.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.90(1).jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.87.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.90.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.89.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.88.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.86.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.85.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.77.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.78.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.79.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.80.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.81.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.82.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.84.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.83.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.84(1).jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.164.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.158.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.159.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.165.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.149.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.161.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.160.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.148.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.162.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.163.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.104.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.110.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.138.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.139.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.105.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.111.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.113.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.107.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.106.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.112.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.116.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.102.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.103.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.117.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.129.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.101.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.115.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.114.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.100.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.128.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.125.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.131.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.119.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.118.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.75.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.130.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.124.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.132.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.126.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.76.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.127.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.133.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.137.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.123.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.99.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.98.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.122.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.136.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.108.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.120.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.134.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.135.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.109.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.121.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.146.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.152.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.153.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.147.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.145.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.151.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.144.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.150.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.154.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.140.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.141.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.155.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.143.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.157.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.156.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.142.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.175.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.166.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.243.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.245.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.246.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.244.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.248.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.239.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.240.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.242.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.241.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.247.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.238.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.237.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.227.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.228.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.229.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.230.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.234.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.232.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.233.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.231.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.236.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.226.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.235.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.225.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.221.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.222.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.220.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.224.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.215.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.217.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.218.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.219.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.216.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.223.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.214.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.213.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.210.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.206.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.205.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.208.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.207.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.209.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.212.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.204.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.203.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.211.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.202.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.201.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.197.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.198.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.196.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.200.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.191.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.192.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.193.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.194.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.195.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.199.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.190.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.189.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.184.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.183.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.185.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.186.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.188.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.179.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.180.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.181.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.182.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.187.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.178.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.177.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.168.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.167.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.169.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.170.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.171.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.173.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.172.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.174.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.176.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.256.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.257.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.258.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.255.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.253.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.252.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.254.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.251.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.250.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.249.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.285.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.279.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.284.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.283.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.274.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.276.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.277.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.278.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.275.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.273.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.272.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.269.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.270.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.271.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.267.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.268.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.263.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.264.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.265.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.266.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.262.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.261.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.259.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.260.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.316.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.317.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.318.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.315.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.314.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.309.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.311.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.312.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.310.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.313.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.307.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.308.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.304.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.305.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.306.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.303.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.302.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.301.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.299.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.300.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.294.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.295.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.296.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.297.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.290.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.292.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.293.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.291.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.298.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.289.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.288.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.287.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.286.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.280.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.281.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.282.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.338.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.330.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.331.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.332.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.336.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.334.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.333.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.335.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.329.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.337.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.327.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.328.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.326.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.325.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.319.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.320.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.321.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.322.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.324.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.323.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.359.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.360.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.356.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.355.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.350.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.352.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.353.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.354.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.357.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.351.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.358.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.349.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.348.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.347.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.346.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.341.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.343.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.340.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.342.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.345.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.339.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.344.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.377.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.378.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.376.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.375.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.374.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.370.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.372.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.371.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.369.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.373.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.368.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.365.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.366.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.364.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.367.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.362.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.363.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.361.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.428.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.430.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.429.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.427.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.435.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.426.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.425.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.421.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.422.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.419.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.424.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.416.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.415.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.418.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.420.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.417.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.423.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.414.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.413.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.409.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.410.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.412.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.411.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.407.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.408.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.401.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.404.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.402.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.403.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.405.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.406.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.400.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.399.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.398.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.397.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.390.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.394.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.391.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.393.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.392.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.389.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.396.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.395.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.385.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.387.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.388.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.386.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.384.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.383.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.380.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.382.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.381.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.379.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.473.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.472.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.464.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.463.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.465.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.470.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.469.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.467.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.466.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.468.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.471.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.462.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.461.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.456.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.457.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.454.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.455.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.460.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.452.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.453.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.458.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.451.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.459.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.450.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.449.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.440.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.441.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.442.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.443.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.446.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.444.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.445.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.439.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.448.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.447.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.438.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.437.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.432.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.431.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.433.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.434.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.436.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.555.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.554.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.550.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.551.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.549.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.553.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.545.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.544.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.547.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.548.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.546.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.552.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.543.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.542.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.537.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.539.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.540.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.538.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.534.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.535.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.536.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.533.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.541.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.532.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.531.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.524.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.525.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.526.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.527.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.528.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.530.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.521.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.522.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.523.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.529.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.520.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.519.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.515.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.517.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.516.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.510.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.511.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.512.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.513.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.514.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.518.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.509.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.508.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.504.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.506.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.507.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.505.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.503.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.502.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.499.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.501.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.500.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.497.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.494.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.496.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.495.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.490.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.491.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.493.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.492.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.498.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.489.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.488.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.487.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.486.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.483.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.485.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.482.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.480.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.479.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.481.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.484.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.474.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.475.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.476.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.477.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.478.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.616.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.617.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.618.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.619.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.615.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.623.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.614.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.613.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.609.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.607.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.610.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.608.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.612.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.603.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.605.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.606.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.604.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.611.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.602.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.601.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.595.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.598.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.597.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.596.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.600.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.592.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.593.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.594.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.591.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.599.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.590.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.589.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.583.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.584.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.586.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.585.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.588.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.580.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.581.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.582.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.579.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.587.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.578.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.577.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.573.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.574.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.575.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.572.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.569.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.568.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.571.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.570.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.576.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.567.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.566.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.560.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.561.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.562.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.563.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.565.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.564.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.556.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.558.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.559.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.557.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.707.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.702.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.703.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.704.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.701.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.700.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.706.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.697.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.699.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.698.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.705.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.696.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.695.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.686.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.687.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.688.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.689.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.690.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.692.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.691.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.685.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.694.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.684.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.693.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.683.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.681.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.678.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.680.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.675.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.674.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.676.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.677.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.679.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.682.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.673.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.672.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.668.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.669.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.667.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.671.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.662.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.664.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.664(1).jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.666.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.665.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.670.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.663.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.661.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.657.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.658.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.660.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.653.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.651.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.654.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.655.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.656.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.652.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.659.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.650.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.649.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.640.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.639.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.641.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.642.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.644.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.643.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.645.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.646.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.648.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.638.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.647.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.637.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.631.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.632.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.633.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.634.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.636.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.628.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.629.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.627.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.630.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.635.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.626.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.625.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.622.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.621.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.620.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.624.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.785.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.784.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.779.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.780.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.781.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.783.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.775.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.776.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.778.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.777.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.774.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.782.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.773.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.772.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.768.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.771.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.761.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.765.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.763.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.764.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.766.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.767.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.762.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.770.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.769.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.760.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.759.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.749.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.750.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.752.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.755.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.753.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.754.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.756.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.751.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.758.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.757.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.748.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.747.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.740.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.742.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.743.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.737.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.738.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.741.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.739.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.746.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.745.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.736.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.744.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.735.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.734.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.726.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.727.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.728.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.729.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.730.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.733.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.723.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.725.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.724.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.732.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.722.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.731.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.721.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.712.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.715.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.713.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.714.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.720.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.716.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.719.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.718.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.710.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.709.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.711.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.708.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.717.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.859.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.861.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.862.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.863.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.864.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.860.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.867.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.858.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.857.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.851.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.852.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.853.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.850.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.854.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.856.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.855.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.847.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.849.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.848.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.846.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.845.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.841.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.839.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.840.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.844.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.843.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.835.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.836.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.837.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.838.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.834.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.842.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.833.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.832.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.825.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.827.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.828.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.829.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.826.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.831.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.824.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.822.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.823.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.830.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.821.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.820.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.817.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.815.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.816.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.819.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.811.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.812.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.814.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.813.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.810.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.818.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.809.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.804.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.803.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.805.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.802.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.808.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.799.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.798.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.800.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.801.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.807.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.806.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.797.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.796.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.792.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.793.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.787.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.786.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.788.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.789.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.790.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.791.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.795.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.794.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.952.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.944.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.943.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.945.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.946.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.947.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.949.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.938.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.951.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.942.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.950.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.941.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.936.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.935.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.934.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.937.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.940.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.931.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.932.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.933.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.930.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.939.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.929.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.928.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.923.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.924.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.925.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.919.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.920.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.921.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.922.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.927.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.918.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.926.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.917.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.916.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.914.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.913.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.908.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.909.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.910.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.911.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.912.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.907.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.915.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.906.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.905.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.901.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.900.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.902.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.899.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.904.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.896.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.897.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.898.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.895.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.903.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.894.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.893.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.884.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.882.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.885.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.886.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.888.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.889.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.887.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.883.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.892.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.891.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.881.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.890.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.880.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.878.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.877.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.876.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.875.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.879.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.872.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.873.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.874.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.871.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.869.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.870.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.866.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.865.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.868.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1037.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1028.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1027.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1015.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1017.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1018.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1019.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1020.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1021.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1022.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1016.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1026.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1025.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1024.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1023.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1014.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1013.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1010.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1009.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1008.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1012.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1003.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1005.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1004.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1007.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1006.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1011.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1002.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1001.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.998.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.997.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.996.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1000.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.991.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.993.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.994.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.995.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.992.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.999.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.990.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.989.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.986.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.985.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.988.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.980.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.982.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.983.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.984.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.981.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.979.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.987.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.978.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.977.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.973.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.974.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.972.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.971.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.976.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.967.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.969.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.970.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.968.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.975.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.966.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.965.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.962.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.961.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.956.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.955.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.958.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.959.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.960.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.964.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.957.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.963.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.954.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.953.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.948.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1058.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1059.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1060.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1057.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1065.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1056.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1055.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1052.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1050.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1051.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1049.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1054.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1045.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1047.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1048.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1046.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1053.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1044.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1043.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1031.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1030.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1032.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1034.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1035.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1036.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1033.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1042.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1041.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1029.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1040.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1039.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1038.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1100.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1099.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1098.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1097.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1096.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1102.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1094.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1095.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1093.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1101.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1092.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1091.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1087.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1090.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1083.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1082.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1084.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1085.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1088.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1086.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1080.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1089.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1081.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1079.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1074.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1076.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1073.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1075.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1078.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1070.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1069.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1071.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1072.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1077.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1068.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1067.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1061.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1063.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1064.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1062.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1066.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1168.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1158.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1167.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1166.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1157.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1156.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1150.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1151.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1153.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1152.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1155.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1147.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1148.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1149.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1146.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1154.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1145.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1144.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1138.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1140.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1139.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1141.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1143.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1135.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1136.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1137.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1134.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1142.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1133.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1132.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1122.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1123.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1124.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1117.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1119.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1120.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1121.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1131.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1118.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1130.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1116.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1129.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1128.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1127.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1126.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1125.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1115.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1110.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1112.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1111.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1114.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1109.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1105.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1106.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1107.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1108.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1113.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1104.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1103.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1172.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1171.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1173.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1174.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1177.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1175.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1176.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1178.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1181.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1180.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1179.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1170.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1169.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1160.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1161.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1164.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1163.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1165.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1162.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1159.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1185.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1184.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1186.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1187.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1189.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1190.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1188.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1191.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1182.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1192.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1193.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1194.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1221.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1213.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1214.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1215.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1216.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1212.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1211.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1207.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1204.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1206.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1208.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1205.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1210.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1201.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1202.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1203.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1209.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1200.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1199.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1188 (1).jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1189 (1).jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1190 (1).jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1191 (1).jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1198.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1185 (1).jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1186 (1).jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1187 (1).jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1184 (1).jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1197.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1196.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1183.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1195.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1304.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1303.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1295.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1296.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1297.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1298.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1299.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1300.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1301.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1294.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1302.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1293.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1292.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1287.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1288.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1289.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1291.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1282.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1284.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1285.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1286.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1283.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1290.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1281.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1280.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1275.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1277.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1276.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1279.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1271.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1270.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1272.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1274.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1273.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1278.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1269.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1268.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1265.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1267.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1259.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1260.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1258.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1261.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1264.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1263.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1262.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1266.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1257.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1248.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1249.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1250.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1251.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1254.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1252.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1253.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1256.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1247.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1255.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1245.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1246.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1241.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1240.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1242.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1244.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1237.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1239.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1236.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1238.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1235.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1243.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1234.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1233.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1225.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1226.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1227.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1228.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1230.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1229.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1231.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1224.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1232.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1223.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1222.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1220.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1218.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1219.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1217.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1399.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1398.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1394.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1395.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1392.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1393.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1397.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1388.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1389.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1390.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1391.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1396.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1387.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1386.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1383.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1382.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1385.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1377.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1376.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1378.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1380.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1381.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1379.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1384.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1375.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1374.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1364.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1366.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1367.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1371.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1369.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1368.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1370.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1365.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1373.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1372.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1363.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1362.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1357.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1358.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1359.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1361.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1352.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1353.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1356.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1355.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1354.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1360.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1351.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1350.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1346.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1347.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1345.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1349.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1342.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1340.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1344.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1343.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1341.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1348.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1339.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1338.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1332.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1333.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1334.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1335.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1337.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1329.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1330.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1331.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1328.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1336.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1327.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1320.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1322.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1321.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1323.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1326.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1316.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1317.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1318.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1319.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1325.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1315.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1324.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1314.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1312.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1309.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1310.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1311.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1313.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1306.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1307.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1308.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1305.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1485.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1487.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1489.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1486.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1493.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1484.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1483.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1478.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1479.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1480.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1477.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1482.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1474.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1475.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1476.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1473.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1481.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1472.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1471.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1464.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1467.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1466.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1465.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1468.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1470.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1461.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1462.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1463.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1469.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1460.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1459.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1456.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1454.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1453.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1455.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1458.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1449.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1451.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1452.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1450.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1457.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1448.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1447.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1441.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1444.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1443.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1442.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1440.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1446.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1438.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1439.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1437.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1445.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1436.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1433.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1430.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1432.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1429.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1435.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1427.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1428.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1431.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1426.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1434.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1425.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1424.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1418.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1419.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1417.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1416.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1423.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1422.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1421.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1412.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1413.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1414.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1415.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1420.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1411.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1410.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1407.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1406.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1404.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1405.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1409.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1400.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1401.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1402.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1403.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1408.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1533.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1532.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1531.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1530.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1529.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1520.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1519.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1515.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1514.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1516.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1513.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1518.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1510.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1511.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1512.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1509.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1517.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1508.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1507.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1502.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1503.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1504.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1506.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1498.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1499.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1501.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1500.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1497.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1505.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1496.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1495.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1488.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1490.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1491.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1492.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1494.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1555.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1554.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1553.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1552.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1551.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1550.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1549.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1548.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1547.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1546.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1545.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1544.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1543.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1542.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1541.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1540.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1539.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1538.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1537.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1536.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1535.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1534.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1587.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1586.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1585.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1584.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1583.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1582.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1581.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1580.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1579.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1578.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1577.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1576.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1575.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1574.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1573.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1572.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1571.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1570.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1561.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1562.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1563.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1569.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1560.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1527.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1528.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1526.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1559.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1558.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1522.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1523.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1524.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1525.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1521.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1557.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1556.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1654.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1653.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1650.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1646.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1651.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1648.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1649.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1647.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1652.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1645.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1644.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1643.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1642(1).jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1638.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1639.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1640.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1642.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1634.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1635.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1636.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1637.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1633.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1641.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1632.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1631.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1625.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1627.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1628.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1626.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1630.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1622.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1624.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1623.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1621.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1629.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1620.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1619.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1614.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1615.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1616.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1618.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1610.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1611.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1612.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1613.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1609.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1617.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1608.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1607.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1604.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1603.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1606.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1598.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1599.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1597.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1600.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1601.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1602.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1605.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1596.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1595.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1568.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1567.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1566.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1565.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1564.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1594.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1593.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1592.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1591.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1590.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1589.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1588.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1703.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1698.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1697.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1700.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1691.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1692.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1694.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1695.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1696.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1693.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1699.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1690.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1689.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1680.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1681.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1682.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1683.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1686.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1684.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1685.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1679.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1688.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1687.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1678.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1677.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1667.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1668.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1669.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1670.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1671.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1673.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1672.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1674.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1676.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1675.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1666.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1665.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1660.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1662.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1661.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1664.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1655.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1656.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1657.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1658.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1659.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1663.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1751.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1753.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1752.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1746.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1747.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1748.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1749.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1750.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1745.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1744.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1738.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1739.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1741.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1740.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1743.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1734.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1735.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1737.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1736.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1742.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1733.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1732.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1728.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1729.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1731.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1722.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1724.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1723.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1726.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1727.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1725.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1730.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1721.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1720.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1712.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1713.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1714.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1715.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1716.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1717.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1719.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1718.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1701.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1711.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1707.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1709.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1710.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1708.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1704.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1702.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1705.jpg
/content/gdrive/My Drive/darknet/build/darknet/x64/data/dog/dog.1706.jpg

Overwriting build/darknet/x64/datadivide/train.txt


In [14]:
%%writefile build/darknet/x64/data/dog.data
classes= 3
train  = build/darknet/x64/datadivide/train.txt
valid  = build/darknet/x64/datadivide/test.txt
names =  build/darknet/x64/data/dog.names
backup = build/darknet/x64/backup/

Overwriting build/darknet/x64/data/dog.data


In [15]:
%%writefile build/darknet/x64/data/dog.names  
dog
left eye
light eye

Overwriting build/darknet/x64/data/dog.names


In [0]:
!pwd
#%pycat build/darknet/x64/data/dog.data
%pycat build/darknet/x64/data/dog.data

/content/gdrive/My Drive/darknet


In [0]:
!ls build/darknet/x64/data/dog

In [0]:
!sudo apt-get install opencv-dev

Reading package lists... Done
Building dependency tree       
Reading state information... Done
E: Unable to locate package opencv-dev


In [0]:
3#!./darknet detector train ./cfg/task/datasets.data cfg/task/yolov3-voc.cfg
#!./darknet detector train cfg/task/datasets.data yolov3-tiny-obj.cfg yolov3-tiny.conv.15
!pwd
!./darknet detector train build/darknet/x64/data/dog.data cfg/yolo-dog.cfg build/darknet/x64/yolov4.conv.137 -dont_show

Streaming output truncated to the last 5000 lines.
v3 (iou loss, Normalizer: (iou: 0.07, cls: 1.00) Region 150 Avg (IOU: 0.580466, GIOU: 0.567488), Class: 0.915777, Obj: 0.007004, No Obj: 0.000625, .5R: 0.636364, .75R: 0.272727, count: 22, class_loss = 2.963174, iou_loss = 5.797876, total_loss = 8.761050 
v3 (iou loss, Normalizer: (iou: 0.07, cls: 1.00) Region 161 Avg (IOU: 0.625813, GIOU: 0.586084), Class: 0.994464, Obj: 0.070035, No Obj: 0.003840, .5R: 0.777778, .75R: 0.111111, count: 18, class_loss = 2.060117, iou_loss = 1.590366, total_loss = 3.650484 

 748: 1.656859, 1.727318 avg loss, 0.000313 rate, 8.947362 seconds, 47872 images, 10.777481 hours left
Loaded: 0.000048 seconds
v3 (iou loss, Normalizer: (iou: 0.07, cls: 1.00) Region 139 Avg (IOU: 0.000000, GIOU: 0.000000), Class: 0.000000, Obj: 0.000000, No Obj: 0.000053, .5R: 0.000000, .75R: 0.000000, count: 1, class_loss = 0.000275, iou_loss = 0.000000, total_loss = 0.000275 
v3 (iou loss, Normalizer: (iou: 0.07, cls: 1.00) Regi

In [8]:
! pwd
!./darknet detector test build/darknet/x64/data/dog.data cfg/yolo-dog.cfg build/darknet/x64/backup/yolo-dog_last.weights build/darknet/x64/data/dog/dog.4.jpg -thresh 0.01! -dont_show  

/content/gdrive/My Drive/darknet
 CUDA-version: 10010 (10010), cuDNN: 7.6.5, GPU count: 1  
 OpenCV version: 3.2.0
 0 : compute_capability = 370, cudnn_half = 0, GPU: Tesla K80 
net.optimized_memory = 0 
mini_batch = 1, batch = 16, time_steps = 1, train = 0 
   layer   filters  size/strd(dil)      input                output
   0 conv     32       3 x 3/ 1    416 x 416 x   3 ->  416 x 416 x  32 0.299 BF
   1 conv     64       3 x 3/ 2    416 x 416 x  32 ->  208 x 208 x  64 1.595 BF
   2 conv     64       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  64 0.354 BF
   3 route  1 		                           ->  208 x 208 x  64 
   4 conv     64       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  64 0.354 BF
   5 conv     32       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  32 0.177 BF
   6 conv     64       3 x 3/ 1    208 x 208 x  32 ->  208 x 208 x  64 1.595 BF
   7 Shortcut Layer: 4,  wt = 0, wn = 0, outputs: 208 x 208 x  64 0.003 BF
   8 conv     64       1 x 1/ 1    208 x 208 x  64 -> 